# Latent Semantic Indexing (LSI)  
LSI is a specific application of dimensionality reduction to the problem of unstructured text processing.
  
At it's core LSI uses a simple Singular Value Decomposition (SVD) on a term-document matrix to perform the dimensionality reduction and map all documents and terms into one shared feature space.  This allows for pairwise conceptual comparison (by simple cosine similarity) of any documents or terms (term-term, term-document, document-document).  The higher the cosine similarity between two terms (or documents), the closer they are in terms of semantic or conceptual meaning.  
  
While the dimensionality reduction idea and term frequency vectors are not new, in it's 26 years of existence LSI has added considerable tradecraft making it (in this author's opinion) the state of the art technique for unstructured text analytics.  

## Building an LSI Space
What does it mean to "build an LSI space"?  This means you've completed the mapping that allows you to represent all documents and terms in the same k-dimensional vector space (where k is chosen).  Importantly, the space is built in such a way that the distance between any term/document vectors represents some measure of the conceptual ("semantic") similarity between them.  
  
The main steps to building an LSI space are:  
<ol>
<li>Procure the Data</li>
<li>Build Term-Document Matrix</li>
<li>Compute SVD</li>
<li>Retain Term and Document Vectors</li>
</ol>

### Procure the Data
The input data to LSI is as simple as can be.  It just requires a set of documents containing unstructured text.  
##### Download the Reuters Data
Download the R52 sample of the ***Reuters 21578*** dataset from [here](http://www.csmining.org/index.php/r52-and-r8-of-reuters-21578.html) and store it locally (or you can just point `read_csv` directly to the download links.  
  
Use pandas to read in the dataset ***r52-train-stemmed*** dataset into a dataframe called `reuters_df_train` with the field names 'cat' and 'text' and read the ***r52-test-stemmed*** dataset into a dataframe called `reuters_df_test` with the same field names.  Make sure you note the value of the `sep` parameter in `read_csv()`!

In [1]:
#Import usual pandas and numpy
import pandas as pd
import numpy as np
# Read in r52 training set with stemming already done for you.
reuters_df_train = pd.read_csv('http://www.csmining.org/tl_files/Project_Datasets/r8%20r52/r52-train-stemmed.txt', sep='\t', names=['cat', 'text'])
# Read in r52 test set with stemming already done for you.
reuters_df_test = pd.read_csv('http://www.csmining.org/tl_files/Project_Datasets/r8%20r52/r52-test-stemmed.txt', sep='\t', names=['cat', 'text'])
# Use head to check out the first few rows of the dataset
reuters_df_train.head()

,cat,text
0,cocoa,bahia cocoa review shower continu week bahia c...
1,earn,champion product approv stock split champion p...
2,acq,comput termin system cpml complet sale comput ...
3,earn,cobanco inc cbco year net shr ct dlr net asset...
4,earn,intern inc qtr jan oper shr loss two ct profit...


##### Download the 20 Newsgroups Dataset
Download the [20 Newsgroups Dataset](http://scikit-learn.org/stable/datasets/twenty_newsgroups.html) with a call to [`sklearn.datasets.fetch_20newsgroups()`](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html#sklearn.datasets.fetch_20newsgroups).  Download the 'train' subset first and store it in `ng_train`, then download the 'test' subset and store it in `ng_test`.

In [2]:
# Import function
from sklearn.datasets import fetch_20newsgroups
# Download train
ng_train = fetch_20newsgroups(subset='train')
# Download test
ng_test = fetch_20newsgroups(subset='test')
type(ng_train)

sklearn.datasets.base.Bunch

##### Examine 20 Newsgroups
Take a look at the attributes of 20 Newsgroups and do the following:  
- Create a `pd.Series` called `ng_train_text` from `ng_train.data`
- Create a `pd.Series` called `ng_test_text` from `ng_test.data`
- Create a `pd.Series` called `ng_train_cats` from `ng_train.target`
- Create a `pd.Series` called `ng_test_cats` from `ng_test.target`

In [3]:
# Training Text
ng_train_text = pd.Series(ng_train.data)
# Training Cats
ng_train_cats = pd.Series(ng_train.target)
# Test Text
ng_test_text = pd.Series(ng_test.data)
# Test Cats
ng_test_cats = pd.Series(ng_test.target)

#### MAKE YOUR CHOICE
Choose between the Reuters dataset for this exercise or the 20 newsgroups dataset at this step.  Whichever one you decide, store the training set text into a Series object called `docs` (make sure you retrieve the right column for Reuters if you choose it.  For the rest of the exercise, you will be working with the dataset you choose here.

In [2]:
## FOR REUTERS
# We're not concerned with the category for now, so select out only the text column into a Series object 'docs'
docs = reuters_df_train['text']
## FOR 20NG
#docs = ng_train_text
# Use head to check out the first few rows of docs
docs.head()

0    bahia cocoa review shower continu week bahia c...
1    champion product approv stock split champion p...
2    comput termin system cpml complet sale comput ...
3    cobanco inc cbco year net shr ct dlr net asset...
4    intern inc qtr jan oper shr loss two ct profit...
Name: text, dtype: object

In [3]:
# Let's load the nltk English stopwords list to ignore those in our analysis
import nltk
from nltk.corpus import stopwords
## Download various nltk corpora (used for stopwords here)
#nltk.download()
## Print all english stopwords
stopwords = stopwords.words('english')

### Build Term-Document Matrix (TDM)
In this step the goal is to encode all of the documents into a matrix where all of the unique terms in the dataset occur along the rows and the documents are the columns.  The values in each entry are some function of the term frequency for that particular term-document intersection.  

There are a variety of different "weightings" that can be applied to the TDM entries, and this is one place where much of the tradecraft improvements of LSI occur.  The overall weighting scheme is generally TFIDF (Term Frequency Inverse Document Frequency) where the total weight is the product of the term frequency and inverse document frequency components:  
\begin{align} w_{ij} = wtf_{ij} \cdot widf_i \end{align}

There are a handful of different term frequency (tf) weighting schemes ranging from binary (1/0 for whether a term occurred or not in the given document), to the actual frequency (count) or the log of the frequency.  For LSI, empirical results have led to the most common tf weighting of:  
\begin{align} wtf_{ij} = log(tf_{ij} + 1) \end{align}  

Similarly there are a handful of different global weighting (idf) schemes ranging from binary to logarithmic to an entropy function that has empirically been found best for LSI:  
\begin{align} widf_{i} = 1 - \sum_j \frac{p_{ij}log(p_{ij})}{log(n)} \end{align}  

In the above equation, n is the number of documents, and p_ij is the term frequency for a given document divided by the term's global frequency:  
\begin{align} p_{ij} = \frac{tf_{ij}}{gtf_i} \end{align}

Given all this, the total weight for each entry is:
\begin{align} w_{ij} = wtf_{ij} \cdot widf_i = log(tf_{ij} + 1) \cdot \left(1 - \sum_j \frac{p_{ij}log(p_{ij})}{log(n)}\right) \end{align}

More details on all the different weightings can be found [here](http://en.wikipedia.org/wiki/Latent_semantic_indexing#Mathematics_of_LSI).

##### Implement TDM Generator
Let's implement a function that takes a Series of documents and generates the matrix with the LSI weightings from above...

In [4]:
'''
The way we're going to attack this is to build out the TDM matrix with the documents as rows and terms as columns
and then we'll call the transpose operator to flip it to the representation we need for LSI.

We need the following:
    1.  Dictionary of word --> index to define vectors (index for each term)
    2.  Dictionary of word --> total count to get the global (IDF)
    3.  Dictionary of word --> document count for each document to get the local (TF) weighting
'''

# Implement a function that returns the 3 dictionaries that we need above
def find_frequencies(docs):
    term_indices = {} ## This is #1 above
    currentIndex = 0 ## This is the counter to make sure we correctly populate the term indices in order
    corpus_bag = {} ## This is #2 above
    doc_bags = [] ## This is the collection for #3 above
    for i, doc in docs.iteritems():
        doc_bag = {} ## This is the dictionary of term frequencies for the doc we're currently examining, doc_bags stores a collection of these
        ## TODO: Tokenize each document with nltk
        doc_tokens = nltk.word_tokenize(doc)
        ## TODO: For each token in the current document:
        for word in doc_tokens:
            ## Optionally ignore stopword and continue
            ## Throw out stopwords
            ##if word in stopwords:
                ##    continue
            ## If the word is new (not in term_indices): 
            if word not in term_indices:
                ## add it to term_indices and give it the index value currentIndex, increment currentIndex
                term_indices[word] = currentIndex
                currentIndex += 1
                ## add it to the corpus_bag with count 1
                corpus_bag[word] = 1
                ## add it to the current doc_bag with count 1
                doc_bag[word] = 1
            ## If the word is not new:
            else:
                ## increment the corpus_bag
                corpus_bag[word] = corpus_bag[word] + 1
                ## If the word is already in the doc_bag, increment that counter, else set it to 1
                if word in doc_bag:
                    doc_bag[word] = doc_bag[word] + 1
                else:
                    doc_bag[word] = 1
        doc_bags.append(doc_bag)
    return term_indices, corpus_bag, doc_bags


Run find_frequencies on docs to return term_indices, corpus_bag, doc_bags

In [5]:
term_indices, corpus_bag, doc_bags = find_frequencies(docs)

Print out the length of term_indices and make sure it is 16144.  Print out term_indices['cocoa'] and make sure it's an int 0-16143

In [6]:
print len(term_indices)
print term_indices['cocoa']
#print term_indices['']

16144
1


Print out the length of corpus_bag and make sure it is 16144.  Print out corpus_bag['cocoa'] and make sure it's 266

In [7]:
print len(corpus_bag)
print corpus_bag['cocoa']

16144
266


Print out the length of doc_bags[0] and make sure it's approximately the length of that document

In [8]:
print len(doc_bags[0])
print doc_bags[0]

135
{'reuter': 1, 'humid': 1, 'routin': 1, 'obtain': 1, 'cruzado': 1, 'held': 1, 'publish': 1, 'kilo': 2, 'superior': 1, 'fit': 1, 'temporao': 2, 'smith': 5, 'late': 1, 'farmer': 2, 'buyer': 1, 'argentina': 1, 'oct': 3, 'earli': 1, 'good': 1, 'march': 4, 'fob': 1, 'arriv': 2, 'consign': 2, 'bean': 2, 'not': 1, 'weekli': 1, 'name': 1, 'reluct': 1, 'level': 1, 'bag': 5, 'februari': 3, 'januari': 1, 'aug': 4, 'bahia': 5, 'view': 1, 'certif': 1, 'liquor': 1, 'juli': 5, 'continu': 1, 'crop': 5, 'export': 2, 'expect': 1, 'year': 2, 'commiss': 1, 'port': 3, 'shower': 1, 'for': 10, 'currenc': 2, 'review': 2, 'per': 3, 'dlr': 14, 'figur': 2, 'estim': 2, 'shipment': 2, 'june': 3, 'sold': 2, 'arroba': 1, 'come': 2, 'sinc': 1, 'stage': 1, 'dry': 1, 'convert': 1, 'shipper': 2, 'zone': 1, 'nearbi': 1, 'april': 4, 'stand': 1, 'rose': 1, 'improv': 1, 'mean': 1, 'sept': 5, 'processor': 1, 'york': 8, 'prospect': 1, 'qualiti': 1, 'period': 2, 'tonn': 2, 'cocoa': 7, 'carniv': 1, 'total': 3, 'open': 2, 'ar

In [9]:
## Useful imports
import math
import scipy
from scipy import linalg

In [13]:
## Implement a function that uses the corpus_bag and doc_bags found above to compute the global weighting (idf) term
def compute_global_weight(corpus_bag, doc_bags):
    global_weights = {} ## A dictionary of term --> global weight (the idf components) using entropy weighting
    ## TODO: Define a variable logn which is the log base 2 of the number of documents in the set
    logn = math.log(len(doc_bags), 2)
    ## TODO: For each doc_bag:
    for doc_bag in doc_bags:
        ## TODO: For each term in the doc_bag
        for term in doc_bag:
            ## TODO: If the term is not in global_weights, initialize it with value 1
            if term not in global_weights:
                global_weights[term] = 1
            ## TODO: Calculate p_ij and increase the term's global weight by p_ij * log(p_ij) / logn
            local_count = doc_bag[term] + 0.0
            global_count = corpus_bag[term]
            pij = local_count/global_count
            global_weights[term] += pij*math.log(pij,2)/logn
    return global_weights

Run compute_global_weight on corpus_bag and doc_bags to generate global_weights

In [14]:
global_weights = compute_global_weight(corpus_bag, doc_bags)

Print global_weights['cocoa'] and make sure it's 0.594249632518

In [15]:
print global_weights['cocoa']

0.594249632518


In [16]:
## Finish the job with a function build_TDM that takes a Series 'docs' and outputs the TDM (a numpy matrix), make it also 
## return the term_indices and global weights as well
def build_TDM(docs):
    ## TODO: Use your first 2 functions from above to populate the term_indices, corpus_bag, doc_bags and global_weights
    term_indices, corpus_bag, doc_bags = find_frequencies(docs)
    global_weights = compute_global_weight(corpus_bag, doc_bags)
    ## TODO: For each doc_bag, generate a doc_vec and add to doc_vecs (these are the "vectors" for each document with weighting)
    doc_vecs = []
    for doc_bag in doc_bags:
        ## TODO: Initialize 'doc_vec' as a list of zeroes with 1 entry per unique term
        doc_vec = [0]*len(corpus_bag)
        ## TODO: For each term in the doc_bag, add the appropriate value into the appropriate place in the doc_vec
        ## NOTE: Need to take advantage of term_indices to get the right index, global_weights and doc_bag to get the value
        for term in doc_bag:
            index = term_indices[term]
            value = global_weights[term]*math.log(doc_bag[term] + 1.0, 2)
            doc_vec[index] = value
        doc_vecs.append(doc_vec)
    ## TODO: Generate a numpy matrix from doc_vecs, and take it's transpose to give the TDM, return that
    tdmatrix = np.matrix(doc_vecs).transpose()
    return term_indices, global_weights, tdmatrix

Now use your build_TDM function on the docs you defined to generate a matrix called 'tdmatrix'

In [17]:
term_indices, global_weights, tdmatrix = build_TDM(docs)

### Compute SVD
Once we have the TDM, the next step is the SVD.  The SVD performs the following decomposition:
\begin{align} X = T\Sigma{D^T} \end{align}
Here X is the TDM, which is a matrix of m terms by n documents.  T is the resultant term space, it has a row for each of the m terms and r column features where r is the <a href='http://en.wikipedia.org/wiki/Rank_(linear_algebra)'>rank</a> of X.  The Sigma matrix is the square diagonal matrix of the r [singular values](http://en.wikipedia.org/wiki/Singular_value) of X in decreasing order.  The final matrix is the transpose of the resulting "document space" so it will be r by n where each column represents a document described by r features.

##### Try it out
Use the linalg svd function with tdmatrix to perform the svd

In [39]:
## Run the svd to yield the full term and document space matrices
## WARNING: This is the computationally intensive step, it will take a long time, so make sure you've taken care of everything before
## this as best as possible so you don't have to do it multiple times
## Once this step is completed, essentially all the computational work is done, you have a trained LSI space!
T,sigma,D_trans = linalg.svd(tdmatrix, full_matrices=False)

#### Reduce the Dimensionality: Rank Reduction
At this point, we haven't really reduced the dimensionality of the problem (all terms and documents have r features where r is probably larger than we want to deal with).  So we make the following approximation:
\begin{align} X \approx T_k\Sigma_kD_k^T \end{align}
Here the first k columns (where k is a chosen parameter) have been selected from the T matrix to yield the m by k matrix T_k.  The singular value matrix has been shrunk to k by k by removing any of the rows or columns greater than k.  The document transpose matrix has been truncated to k by n by selecting just the first k rows of the matrix.

The mathematics of the SVD tell us that this approximation is the best possible rank k approximation to X that we can possibly make.  Thus, by then using those matrices we have performed dimensionality reduction to k dimensions.

##### Try it Out
Generate 3 matrices, T_k, D_trans_k, and sigma_inv_k by performing the rank-reduction approximation.

In [40]:
## Truncate the resulting matrices to dimension k (you select this dimension, higher values retain more information at complexity cost)
k = 100
m = T.shape[0]
n = D_trans.shape[1]
T_k = T[0:m, 0:k]
print T_k.shape
D_trans_k = D_trans[0:k, 0:n]
print D_trans_k.shape
sigma_inv = np.linalg.inv(linalg.diagsvd(sigma, n, n))
sigma_inv_k = sigma_inv[0:k, 0:k]
print sigma_inv_k.shape

(16144, 100)
(100, 6532)
(100, 100)


### Comparing Term and Document Vectors
At this point, any of the terms can be plucked from the rows of the k-dimensional T_k matrix and compared to one another for conceptual similarity.  Similarly, the same can be done for any document to document comparisons via the columns of the D_trans_k matrix.  

#### Folding Document Vectors
However, the one aspect that is missing is the ability to make comparisons between the two, aka we want to map the documents into the term space so that we can compare terms to documents.  The transformation that does this is:
\begin{align} D_k = X^TT_k\Sigma_k^{-1} \end{align}

What this means, is that given any new document vector d we can "fold it in" to the feature space by giving the vector the appropriate weightings (TFIDF) in the X space and then multiplying it by the matrices T_k and Sigma_inv_k.

##### Try it out:
Implement a function fold_doc that takes a blob of text 'doc_text', term_indices, global_weights, T_k, and sigma_inv_k and returns a k dimension vector in the shared term-document space via document folding.

In [41]:
## Function that folds a new document into an existing LSI space (space designated by global weightings, term indices, and T_k and sigma_inv_k)
def fold_doc(doc_text, term_indices, global_weights, T_k, sigma_inv_k):
    tokens = nltk.word_tokenize(doc_text)
    doc_bag = {}
    for token in tokens:
        if token in doc_bag:
            doc_bag[token] = doc_bag[token] + 1
        else:
            doc_bag[token] = 1
    a_vec = [0]*len(term_indices)
    for term in doc_bag:
        if term in term_indices:
            index = term_indices[term]
            a_vec[index] = global_weights[term]*math.log(doc_bag[term] + 1.0, 2)
    a = np.matrix(a_vec)
    folded_vec = np.dot(np.dot(a, T_k), sigma_inv_k)
    return folded_vec

#### Cosine Similarity
The way we will compare vectors in our LSI space is via [cosine similarity](http://en.wikipedia.org/wiki/Cosine_similarity).  The equation for it is as follows:
\begin{align} \cos {(a,b)} = \frac{a \cdot b}{\|{A}\|\|{B}\|} \end{align}

Essentially, it is the normalized dot (scalar) product between 2 vectors.  This is the defacto standard for distance metric in LSI.

##### Try it out
Write a function cosine_sim that takes 2 vectors and returns their cosine similarity score

In [42]:
def cosine_sim(a, b): 
    return linalg.norm(np.dot(a/linalg.norm(a), b.transpose()/linalg.norm(b)))

Try out your function with the first few Reuters documents

In [22]:
doc1 = docs[0]
doc2 = docs[2]
vec1 = fold_doc(doc1, term_indices, global_weights, T_k, sigma_inv_k)
vec2 = fold_doc(doc2, term_indices, global_weights, T_k, sigma_inv_k)
print cosine_sim(vec1, vec2)

0.0976533754888


## Reuters Document Categorization with LSI
Now that we've built our LSI space and we know how to fold in new documents into the space, we can try to perform the Reuters text categorization task.  Our approach will be a simple one, yet perform surprisingly well.  We will simply take the test set and for each document find the closest (by cosine similarity) document to it in the training set, then assign the test document to that training document's assigned category.   

This is a simple manual KNN classifier, with k=1 and cosine similarity as a distance metric.  We can of course try different metrics and values of k if you like.

##### Try it out
Build a function classify_docs that takes a dataframe of documents with columns 'cat' and 'text' does the following:
<ul>
<li>Prints the actual category and the predicted category for each document</li>
<li>Prints the total docs tested</li>
<li>Prints the total docs correct</li>
<li>Prints the overall accuracy of prediction</li>
</ul>

In [43]:
## Classify documents
def classify_docs(df_test, term_indices, global_weights, T_k, sigma_inv_k):
    test_doc_count = 0
    correct_count = 0
    ## TODO: iterate through the rows of df_test using iterrows
    for row in df_test.iterrows():
        ## TODO: Retrive the actual cat and test for each row
        test_cat = row[1]['cat']
        test_doc = row[1]['text']
        ## TODO: Fold the test document into the space to give it a vector
        folded_vec = fold_doc(test_doc, term_indices, global_weights, T_k, sigma_inv_k)
        ## TODO: Compare the resultant vectors via cosine similarity and give each test document the category 
        ## of the training document closest to it.  Print out the right and predicted categories.  Keep track of right/wrong
        best_score = -1.0
        record = None
        index = 0
        for j in xrange(0, D_trans_k.shape[1]):
            training_vec = D_trans_k[0:k, j:j+1]
            score = np.dot(folded_vec, training_vec)/linalg.norm(folded_vec)/linalg.norm(training_vec)
            if score > best_score:
                best_score = score
                record = reuters_df_train[j:j+1]
                index = j
        found_cat = record.cat[index]
        print test_cat + "," + found_cat
        test_doc_count += 1
        if test_cat==found_cat:
            correct_count += 1
    ## TODO: Print out the summary results
    print "Total Docs Test: " + str(test_doc_count)
    print "Total Correct: " + str(correct_count)
    print "Accuracy: " + str((correct_count+0.0)/test_doc_count)
    return

Load the R52 Stemmed Test set into a dataframe and try out the categorization.

In [44]:
df_test = reuters_df_test
classify_docs(df_test, term_indices, global_weights, T_k, sigma_inv_k)

trade,trade
grain,grain
ship,ship
gold,gold
acq,acq
tin,tin
ipi,ipi
earn,earn
earn,earn
acq,earn
jobs,earn
earn,earn
earn,crude
earn,earn
earn,earn
tin,tin
trade,trade
zinc,tin
sugar,sugar
sugar,sugar
acq,acq
earn,earn
acq,acq
earn,earn
trade,trade
acq,acq
acq,iron-steel
earn,earn
trade,trade
cpi,crude
acq,acq
sugar,sugar
sugar,sugar
earn,earn
nickel,nickel
earn,earn
acq,acq
earn,earn
earn,earn
acq,acq
acq,acq
acq,acq
earn,earn
ship,ship
earn,earn
ship,ship
earn,earn
earn,earn
earn,earn
earn,earn
earn,earn
earn,earn
earn,earn
earn,earn
earn,earn
jobs,jobs
earn,earn
acq,acq
earn,earn
acq,acq
earn,earn
earn,earn
earn,earn
sugar,sugar
money-fx,money-fx
sugar,sugar
earn,earn
earn,earn
earn,earn
earn,earn
acq,acq
earn,earn
acq,crude
earn,earn
earn,earn
earn,earn
earn,earn
sugar,sugar
earn,earn
sugar,sugar
earn,earn
earn,earn
acq,acq
earn,earn
earn,earn
earn,earn
earn,earn
acq,acq
earn,earn
acq,acq
heat,heat
jobs,jobs
earn,earn
money-fx,money-fx
interest,interest
acq,acq
earn,earn
earn,earn
